# Preparando los datos de pobreza

Le meta de este notebook es leer los datos de pobreza de la hoja oficial de Excel y crear un .csv con todos los estados en un solo archivo.

In [3]:
import pandas as pd
import geopandas as gpd
from zipfile import ZipFile
from pathlib import Path
from datetime import datetime as dt
%matplotlib inline

hoy = dt.today().strftime("%d-%b-%y")

hoy

'24-Jun-19'

In [8]:
DATOS_BRUTOS = Path("../../datos/brutos/")
DATOS_INTERINOS = Path("../../datos/interinos/")
DATOS_PROCESADOS = Path("../../datos/procesados/")
DATOS_EXTERNOS = Path("../../datos/externos/")

## Datos pobreza

In [9]:
ZipFile(DATOS_BRUTOS / 'Base de datos de pobreza AGEB segun entidad federativa 2015.zip').extractall(DATOS_INTERINOS / 'datos_pobreza')

In [11]:
archivo_excel = pd.ExcelFile(DATOS_INTERINOS / 'datos_pobreza' / 'Base de datos de pobreza AGEB segun entidad federativa 2015.xlsx')

In [12]:
archivo_excel.sheet_names

['Índice',
 'Aguascalientes',
 'Baja California  ',
 'Baja California Sur ',
 'Campeche',
 'Coahuila',
 'Colima',
 'Chiapas',
 'Chihuahua',
 'Ciudad de México',
 'Durango',
 'Guanajuato',
 'Guerrero',
 'Hidalgo',
 'Jalisco',
 'México',
 'Michoacán',
 'Morelos',
 'Nayarit',
 'Nuevo León',
 'Oaxaca',
 'Puebla',
 'Querétaro',
 'Quintana Roo',
 'San Luis Potosí',
 'Sinaloa',
 'Sonora',
 'Tabasco',
 'Tamaulipas',
 'Tlaxcala',
 'Veracruz',
 'Yucatán',
 'Zacatecas']

In [13]:
pestañas_archivos = archivo_excel.sheet_names[1:].copy()

In [19]:
pestañas_archivos

['Aguascalientes',
 'Baja California  ',
 'Baja California Sur ',
 'Campeche',
 'Coahuila',
 'Colima',
 'Chiapas',
 'Chihuahua',
 'Ciudad de México',
 'Durango',
 'Guanajuato',
 'Guerrero',
 'Hidalgo',
 'Jalisco',
 'México',
 'Michoacán',
 'Morelos',
 'Nayarit',
 'Nuevo León',
 'Oaxaca',
 'Puebla',
 'Querétaro',
 'Quintana Roo',
 'San Luis Potosí',
 'Sinaloa',
 'Sonora',
 'Tabasco',
 'Tamaulipas',
 'Tlaxcala',
 'Veracruz',
 'Yucatán',
 'Zacatecas']

Nota: Cada pestaña tiene la tabla de un estado. Los encabezados estan en la línea 5 pero los datos no comienzan hasta la línea 8.

In [20]:
estados = pd.concat((archivo_excel.parse(estado, header = 4, usecols = ['Clave de \nAGEB', 'Entidad \nfederativa', 'Municipio', 'Rango de pobreza (%)', 'Rango de pobreza extrema (%)']) for estado in pestañas_archivos), ignore_index = True)

In [21]:
estados.head()

,Entidad federativa,Municipio,Clave de AGEB,Rango de pobreza (%),Rango de pobreza extrema (%)
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,Aguascalientes,Aguascalientes,0100100010229,"[ 0, 18]","[ 0, 20]"
3,Aguascalientes,Aguascalientes,0100100010233,"(34, 50]","[ 0, 20]"
4,Aguascalientes,Aguascalientes,0100100010286,"[ 0, 18]","[ 0, 20]"


In [22]:
estados.dropna(axis = 1, how = 'all', inplace = True)
estados.dropna(how = 'any', inplace = True)

estados.columns = ['Entidad Federativa', 'Municipio', 'CVEGEO',  'Rango de pobreza (%)', 'Rango de pobreza extrema (%)']

estados.head()

,Entidad Federativa,Municipio,CVEGEO,Rango de pobreza (%),Rango de pobreza extrema (%)
2,Aguascalientes,Aguascalientes,0100100010229,"[ 0, 18]","[ 0, 20]"
3,Aguascalientes,Aguascalientes,0100100010233,"(34, 50]","[ 0, 20]"
4,Aguascalientes,Aguascalientes,0100100010286,"[ 0, 18]","[ 0, 20]"
5,Aguascalientes,Aguascalientes,0100100010290,"[ 0, 18]","[ 0, 20]"
6,Aguascalientes,Aguascalientes,0100100010303,"(18, 34]","[ 0, 20]"


In [23]:
estados.to_csv(DATOS_PROCESADOS / 'agebs.csv', index = False, encoding = 'utf-8')